In [62]:
file = open('demofile.dots', 'r', encoding='UTF-8')

In [63]:
lines = file.readlines()

In [55]:
import re
declarator = r":[_A-Za-z][A-Za-z0-9]*:"
functional_symbols = [
    {
        "name":"open_parentheses",
        "matcher":r"\("
    },
    {
        "name":"close_parentheses",
        "matcher":r"\)"
    },
    {
        "name":"open_curly",
        "matcher":r"\{"
    },
    {
        "name":"close_curly",
        "matcher":r"\}"
    }
]
variable_types = [
    {
        "converts_to":str,
        "matcher":r"string "
    }
]



In [64]:
from operator import indexOf
import re
stack = []
for line in lines:
    found_declarator = re.search(declarator,line)
    if found_declarator:
        stack.append(
            {
                'line': indexOf(lines,line),
                'position': found_declarator.span(),
                'match': found_declarator.group()
            }
        )
    for functional_symbol in functional_symbols:
        found_functional_symbol = re.search(functional_symbol["matcher"], line)
        if found_functional_symbol:
            stack.append(            {
                'line': indexOf(lines,line),
                'position': found_functional_symbol.span(),
                'match': found_functional_symbol.group()
            })
            
print(stack)

[{'line': 0, 'position': (1, 4), 'match': ':m:'}, {'line': 0, 'position': (0, 1), 'match': '('}, {'line': 0, 'position': (5, 6), 'match': ')'}, {'line': 0, 'position': (6, 7), 'match': '{'}, {'line': 2, 'position': (4, 7), 'match': ':p:'}, {'line': 2, 'position': (7, 8), 'match': '('}, {'line': 2, 'position': (13, 14), 'match': ')'}, {'line': 7, 'position': (0, 1), 'match': '}'}]


In [68]:
for item in stack:
    print(str(lines[item['line']])[item['position'][0]:item['position'][1]])

:m:
(
)
{
:p:
(
)
}
